# Activity Balance

In [1]:
import pandas as pd
import networkx as nx
import joblib 
% pylab inline
import glob
import scipy
import random
import itertools
from collections import Counter
from statsmodels.stats.weightstats import DescrStatsW
import os


import sys
sys.path.insert(0, '../Script/library3/')

from utilities import *
from burst_func import *
from data_clean_save import *
from randomization import *
from plots import *
from tqdm import tqdm

Populating the interactive namespace from numpy and matplotlib


### Main Method

Take a temporal network in form of a df, the line graph, a time dt and compute, for each time series of an egonetwork the ratio of contacts that involve either one or the other node connected by the central link.

In [2]:
def compute_activity_balance(df,line_graph,dt,title1):
        

        node_list = list(g.nodes)
        df_list = list()
    
       
        cnt1_n_links = Counter()
        
        
       
        k = 0
        for nodes in tqdm(df.nodes.unique()):
            
            
            
            
            neigh_set = (set(nx.neighbors(g,nodes)))
            
            df_rest_neigh = df[df.nodes.isin(neigh_set)]  

            
            brst_cnt_neigh,df_rest_neigh = get_burst_train(df_rest_neigh,dt)
            n1,n2 = nodes
                        
            df_rest_neigh_tot = df_rest_neigh.groupby('burst')['nodes'].apply(lambda x:(float(len(set(x))),float(len(list(x)))))
            n1_df = df_rest_neigh[df_rest_neigh.nodes.apply(lambda x: n1 in x)].groupby('burst')['nodes'].apply(lambda x:(float(len(set(x))),float(len(list(x)))))
            n2_df = df_rest_neigh[df_rest_neigh.nodes.apply(lambda x: n2 in x)].groupby('burst')['nodes'].apply(lambda x:(float(len(set(x))),float(len(list(x)))))
            
            if n1_df.shape[0] == 0: 
                 cnt1_n_links_node = Counter(n2_df.apply(lambda x:(x,(0,0),x)).values)
                
            elif n2_df.shape[0] == 0:
                 cnt1_n_links_node = Counter(n1_df.apply(lambda x:(x,x,(0,0))).values)
            
            else:
                df_rest_neigh_tot = pd.concat([df_rest_neigh_tot,n1_df,n2_df],axis = 1)
                df_rest_neigh_tot.columns = ['tot','n1','n2']
                cnt1_n_links_node = Counter(df_rest_neigh_tot.apply(lambda x:(x.tot,x.n1,x.n2),axis = 1).values)
            

            cnt1_n_links.update(cnt1_n_links_node)
            
        return cnt1_n_links

### Example

In [6]:
title1 = 'tij_lnVS'
dt = 60
df = get_df(title1)
g = get_linegraph(title1)
compute_activity_balance(df,g,dt,title1)

100%|██████████| 755/755 [01:19<00:00,  9.45it/s]


Counter({((8.0, 22.0), (3.0, 10.0), (5.0, 12.0)): 1,
         ((4.0, 8.0), (2.0, 5.0), (2.0, 3.0)): 1,
         ((3.0, 16.0), (2.0, 7.0), (1.0, 9.0)): 1,
         ((2.0, 48.0), (1.0, 2.0), (1.0, 46.0)): 1,
         ((3.0, 11.0), (2.0, 9.0), (1.0, 2.0)): 2,
         ((3.0, 6.0), (1.0, 4.0), (2.0, 2.0)): 1,
         ((6.0, 18.0), (1.0, 1.0), (5.0, 17.0)): 1,
         ((2.0, 24.0), (1.0, 23.0), (1.0, 1.0)): 2,
         ((5.0, 10.0), (1.0, 3.0), (4.0, 7.0)): 1,
         ((3.0, 51.0), (1.0, 7.0), (2.0, 44.0)): 1,
         ((2.0, 11.0), (1.0, 6.0), (1.0, 5.0)): 1,
         ((5.0, 17.0), nan, (5.0, 17.0)): 15,
         ((7.0, 22.0), (4.0, 18.0), (3.0, 4.0)): 1,
         ((2.0, 11.0), (1.0, 2.0), (1.0, 9.0)): 5,
         ((1.0, 15.0), (1.0, 15.0), nan): 55,
         ((4.0, 6.0), (1.0, 2.0), (3.0, 4.0)): 2,
         ((2.0, 23.0), (1.0, 21.0), (1.0, 2.0)): 1,
         ((1.0, 3.0), nan, (1.0, 3.0)): 3446,
         ((5.0, 5.0), (3.0, 3.0), (2.0, 2.0)): 1,
         ((4.0, 10.0), (3.0, 5.0), (1.0, 5

### Compute for dt = 60 and each dataset

In [ ]:


try:os.mkdir('../Results/Bursty_trains/activity_balance')
except:
    print 'folder already existing'

for dt in [60]:
    cnt_tot = {}
    link_cnt_tot = {}

    for title1 in title_list_plot:
        print title1
        df = get_df(title1)
        g = get_linegraph(title1)

        link_cnt_tot[title1+str(dt)] = compute_activity_balance(df,g,dt,title1)
        
    joblib.dump(link_cnt_tot,'../Results/Bursty_trains/activity_balance/activity_balance_'+str(dt)+'.joblib')
    

## Plots

In [6]:
def get_df_cnt(cnt):

    df_cnt = pd.DataFrame(data=cnt.keys(),index = cnt.values()).reset_index()
    df_cnt.columns = ['cnt','tot','n1','n2']

    df_cnt['n1_r'] = df_cnt['n1'].apply(lambda x: (0,0) if isnan(x).any() else x)
    df_cnt['n2_r'] = df_cnt['n2'].apply(lambda x: (0,0) if isnan(x).any() else x)

    df_cnt['norm'] = df_cnt['n1_r'].apply(lambda x:x[1]) + df_cnt['n2_r'].apply(lambda x:x[1])
    df_cnt['burst_size'] = df_cnt['tot'].apply(lambda x:x[1]) 

    df_cnt['balance'] = df_cnt.apply(lambda x: max(x.n1_r[1],x.n2_r[1])/x.norm, axis = 1)

    df_cnt['weight'] = df_cnt['balance'] * df_cnt['cnt']

    norm = df_cnt.groupby('burst_size')['cnt'].sum()

    return df_cnt, (df_cnt.groupby('burst_size')['weight'].sum()/norm)

In [ ]:





max_cumsum = 0.95
max_dic = {}
color_dic = {}
color_dic['60'] = 'b'

for k,group in enumerate([ mail_list, human_contact_list]):
    
    if group in [human_contact_list]:fig,axes = plt.subplots(ncols = 3,nrows = 3,figsize= (23,15) )
    else: fig,axes = plt.subplots(ncols = 2,nrows = (len(group)//2) + (len(group)%2),figsize= (14,10) )
    
    plt.subplots_adjust(hspace = 0.3)
    plt.subplots_adjust(wspace = 0.3)
    str1 = 'group'+str(k)
    
    for dt in ['60',]:
        link_cnt_tot = joblib.load('../Results/Bursty_trains/activity_balance/activity_balance_60.joblib')

        for i,title1 in enumerate(group):
            print title1 + s + dt
            ax = axes.flatten()[i]
            ax.tick_params(axis='both', which='major', labelsize=14)
            ax.tick_params(axis='both', which='minor', labelsize=14)
            df = get_df_cnt(link_cnt_tot[title1+dt])[0]
            df = df[df.burst_size>1]
            sr = df.groupby('balance')['cnt'].sum()

            sr.sort_index(inplace=True)
            sr = sr/float(sum(sr))
            mean1 = np.average(sr.index,weights=sr.values)
            sr = (sr).cumsum()
            if central_link: 
                label1 = r'$\Delta t = $'+ dt + ', central link'
                linestyle1 = '--'
            else: 
                label1 = r'$\Delta t = $'+ dt + ''
                linestyle1 = '-'
#                 try: 
#                     if max_dic[title1]< sr.iloc[-2]: max_dic[title1] = sr.iloc[-2]
#                 except: max_dic[title1] = sr.iloc[-2]
            max_idx = max(sr.index[sr.index<=max_cumsum])
            max_dic[title1] = sr[max_idx]
            print r'$P[\mathcal{B}<=0.95] = $'+str(max_dic[title1])
            print r'$E[\mathcal{B}]$ = '+str(mean1)

            ax.plot(sr,label = label1,linestyle = linestyle1, color = color_dic[dt])
            ax.legend()
            ax.set_title(title_to_paper_name[title1],size = 20)
            ax.set_yscale('log')
            ax.set_xlabel('x',size=20)
            ax.set_ylabel(r'$Pr[\mathcal{B}\leq x]$',size = 20)
            print title1 + s + dt
            ax.hlines(max_dic[title1],0.5,1,linestyle = ':',label = r'$P[\mathcal{B} \leq $'+ ' '+ str(max_cumsum) + '$ ] = $ ' + str(round(max_dic[title1],2)))
            ax.hlines(mean1,0.5,1,linestyle = '--',label = r'$E[\mathcal{B}]$ = '+ ' '+ str(round(mean1,2)))
            ax.legend()
    #plt.savefig('../plot_results/bursty_train_activity_balance_full_prepr/activity_balance_cum_sum'+str1+'_10_new_datasets.eps',format = 'eps')
    plt.show()